In [147]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D,UpSampling1D
from keras.models import Model
from keras import backend as K
from keras import metrics

# Setup the network parameters:
beta = 4
n_features = 4
boat_csv = pd.read_csv("Data/Boat_nominal_data/Boat_nominal_sequences.csv")
boat_csv = boat_csv.drop(columns=["Unnamed: 0","Speed","Acceleration","M0C","M1C"])
scaler = StandardScaler()
normal_data = scaler.fit_transform(boat_csv)
print(normal_data.shape)

boat_val = pd.read_csv("Data/Boat_nominal_data/Boat_nom_validation.csv")
boat_val = boat_val.drop(columns=["Unnamed: 0","Speed","Acceleration","M0C","M1C"])
scaler = StandardScaler()
val_nom_data = scaler.fit_transform(boat_val)


boat_anom_csv = pd.read_csv("Data/Boat_anomalous_big.csv")
boat_anom_csv= boat_anom_csv.drop(columns=["Unnamed: 0"])    
scaler = StandardScaler()
anomalous_data = scaler.fit_transform(boat_anom_csv)
print(anomalous_data.shape)

boat_anom_val_csv = pd.read_csv("Data/Boat_anomalous_validation.csv")
boat_anom_val_csv= boat_anom_val_csv.drop(columns=["Unnamed: 0"])    
scaler = StandardScaler()
anomalous_val_data = scaler.fit_transform(boat_anom_val_csv)
print(anomalous_val_data.shape)

def prepare_sequences(data, batch_size):
    samples = []
    for i in range(0,data.shape[0]- batch_size, batch_size):
        sample = data[i:i+batch_size]	
        samples.append(sample)

    sequences = np.array(samples)

    # Batch size (Number of samples time steps and number of features
    trainX = np.reshape(sequences, (len(sequences), batch_size, n_features))

    return trainX


def prepare_data():    
    trainX_nominal = prepare_sequences(normal_data,1024) 
    print(trainX_nominal.shape)
    
    valX_nominal = prepare_sequences(val_nom_data,1024)
    print(valX_nominal.shape)
    
    # trainX_anomalous = prepare_sequences(anomalous_data,6600)
    # print(trainX_anomalous.shape)  
    # 
    # valX_anom = prepare_sequences(anomalous_val_data,6600)
    # print(valX_anom.shape)

    return trainX_nominal, valX_nominal#, trainX_anomalous, valX_anom



trainX_nominal, valX_nominal = prepare_data()


(308224, 4)


(72600, 8)
(13200, 8)
(300, 1024, 4)
(10, 1024, 4)


In [148]:
from keras import objectives
from keras.layers import RepeatVector, MaxPooling1D

input_shape = (1024, n_features)
kernel_size = 7
filters = 64
latent_dim = 4
epochs = 30
use_mse = True
load_weights = False


def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


inputs = Input(shape=input_shape, name='encoder_input')
x = inputs
for i in range(4):
    x = Conv1D(filters=filters,
               kernel_size=7,
               padding='same')(x)
    x = MaxPooling1D(2)(x)
    filters = int(filters / 2)


shape = K.int_shape(x)

x = Flatten()(x)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(shape[1] * shape[2], name='Dense_after_sampling')(latent_inputs)
x = Reshape((shape[1], shape[2]))(x)
filters = filters * 2

for i in range(4):
    x = Conv1D(filters=filters,kernel_size=7, padding='same')(x)
    x = UpSampling1D(size=2)(x)
    filters = filters * 2
    
    
outputs = Conv1D(filters=n_features, kernel_size=7, padding='same')(x)


decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae')

from keras.losses import mse

reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var))
loss = reconstruction_loss + beta*kl_loss
vae.add_loss(loss)

vae.compile(optimizer='rmsprop', metrics= ['accuracy'])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 1024, 4)      0                                            
__________________________________________________________________________________________________
conv1d_301 (Conv1D)             (None, 1024, 64)     1856        encoder_input[0][0]              
__________________________________________________________________________________________________
max_pooling1d_135 (MaxPooling1D (None, 512, 64)      0           conv1d_301[0][0]                 
__________________________________________________________________________________________________
conv1d_302 (Conv1D)             (None, 512, 32)      14368       max_pooling1d_135[0][0]          
__________________________________________________________________________________________________
max_poolin

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z_sampling (InputLayer)      (None, 4)                 0         
_________________________________________________________________
Dense_after_sampling (Dense) (None, 512)               2560      
_________________________________________________________________
reshape_33 (Reshape)         (None, 64, 8)             0         
_________________________________________________________________
conv1d_305 (Conv1D)          (None, 64, 8)             456       
_________________________________________________________________
up_sampling1d_135 (UpSamplin (None, 128, 8)            0         
_________________________________________________________________
conv1d_306 (Conv1D)          (None, 128, 16)           912       
_________________________________________________________________
up_sampling1d_136 (UpSamplin (None, 256, 16)           0         
__________

In [149]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath="Models/Weights/weights.hdf5", verbose=1, save_best_only=True)

vae.fit(x=trainX_nominal, epochs=100, 
        batch_size=1024,
        validation_data=(valX_nominal,None),
        callbacks=[checkpointer])
vae.load_weights('Models/Weights/weights.hdf5')

# vae.save("Models/Conv1d_VAE_comp.h5")

Train on 300 samples, validate on 10 samples
Epoch 1/100


300/300 [==============================] - 20s 66ms/step - loss: 1.4620 - val_loss: 21.6330



Epoch 00001: val_loss improved from inf to 21.63301, saving model to Models/Weights/weights.hdf5


Epoch 2/100


300/300 [==============================] - 2s 6ms/step - loss: 21.6570 - val_loss: 4.0090


...


Epoch 00098: val_loss did not improve from 0.13987
Epoch 99/100


300/300 [==============================] - 2s 7ms/step - loss: 0.1738 - val_loss: 0.1801



Epoch 00099: val_loss did not improve from 0.13987
Epoch 100/100


300/300 [==============================] - 2s 6ms/step - loss: 0.1807 - val_loss: 0.2173



Epoch 00100: val_loss did not improve from 0.13987


In [119]:
import sys

orig_stdout = sys.stdout
f = open('Logs/out.txt', 'w')
sys.stdout = f
print(encoder.summary())
print(decoder.summary())
sys.stdout = orig_stdout
f.close()


In [150]:



nom_autoenc = vae.predict(trainX_nominal)

autoenc_df = pd.DataFrame(nom_autoenc[0], columns=boat_csv.columns)

plt.plot(boat_csv['Lon'][:1024], boat_csv['Lat'][:1024])
plt.show()
plt.plot(autoenc_df['Lon'], autoenc_df['Lat'])
plt.show()





In [151]:
nom_enc = encoder.predict(trainX_nominal)
print(len(nom_enc), nom_enc[0].shape)

titles = ["Mean","Sigmas","Sampled"]

for i, elem in enumerate(nom_enc):
    plt.plot(elem)
    plt.title(titles[i])
    plt.show()








3 (300, 4)
